# Vehicle Diagnostic Application Emulation (Low-Level CAN Communication)

This notebook handles the **raw CAN socket communication** used to retrieve a vehicle's VIN using the SAE J1939 protocol.

It includes:
- A function to send a VIN request using the appropriate PGN.
- A function to receive the response using the BAM (Broadcast Announce Message) protocol.
- A wrapper function that ties both together.

This notebook is meant to be imported (or `%run`) by higher-level components like `DLL_layer.ipynb`.

In [ ]:
import socket
import struct

# Name of the CAN interface (must match your system)
CAN_INTERFACE = "can1"

# Format for CAN frame: 4 bytes CAN ID, 1 byte DLC, 3 bytes padding, 8 bytes data
can_frame_fmt = "=LB3x8s"

## 📤 Sending a VIN Request

We send a request for PGN `0xFEEC` (Request for VIN) to the address `0x00` or Engine.

In [1]:
def send_vin_request(sock):
    # CAN ID: PGN 0xEA00 + destination address 0xF9 (global) + EFF flag
    can_id = 0x18EA00F9 | socket.CAN_EFF_FLAG  # 29-bit CAN ID (Extended Frame Format)

    # Data field for PGN 0xFEEC (Request VIN)
    data = bytes([0xEC, 0xFE, 0x00])  # Least significant byte first

    # Pack the CAN frame (ID, DLC=3, and padded data to 8 bytes)
    frame = struct.pack(can_frame_fmt, can_id, 3, data.ljust(8, b'\x00'))

    # Send the CAN frame on the socket
    sock.send(frame)

## 📥 Receiving VIN via BAM

The Engine sends the VIN using the Broadcast Announce Message (BAM) transport protocol.
- It first sends a control packet with the total length and packet count.
- Then it sends multiple data packets, each containing a chunk of the VIN.

We collect all data packets until the full VIN is received.

In [2]:
def receive_bam_vin(sock):
    vin_bytes = bytearray()
    total_length = 0
    expected_packets = 0
    collecting = False
    received_packets = 0
    expected_src = 0x00  # Assuming VIN comes from source address 0x00 (can be made dynamic)

    while True:
        cf, _ = sock.recvfrom(16)  # Receive raw CAN frame
        can_id, dlc, data = struct.unpack(can_frame_fmt, cf)
        arb_id = can_id & 0x1FFFFFFF  # 29-bit arbitration ID

        # Extract PGN and source address from CAN ID
        pgn = (arb_id >> 8) & 0xFFFF
        src = arb_id & 0xFF

        # Handle the BAM control packet
        if pgn == 0xECFF and data[0] == 0x20 and src == expected_src and data[5:8] == bytes([0xEC, 0xFE, 0x00]):
            total_length = data[1]
            expected_packets = data[2]
            collecting = True
            vin_bytes = bytearray()
            received_packets = 0

        # Handle data packets
        elif pgn == 0xEBFF and collecting and src == expected_src:
            seq_num = data[0]
            payload = data[1:8]
            vin_bytes += payload
            received_packets += 1

            # Stop if full VIN is collected
            if len(vin_bytes) >= total_length:
                break

    return vin_bytes[:17].decode('ascii', errors='ignore')  # Decode VIN from collected bytes

## 🔁 Wrapper: Send + Receive VIN

This function wraps both the request and response logic into a single call.
It opens the CAN socket, sends the request, waits for the response, and returns the decoded VIN.

In [ ]:
def get_vin_from_can():
    # Open a raw CAN socket and bind to interface
    can_sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
    can_sock.bind((CAN_INTERFACE,))

    # Perform request and receive
    send_vin_request(can_sock)
    vin = receive_bam_vin(can_sock)

    can_sock.close()
    return vin

## 🧪 Optional Test

Uncomment the line below to run this notebook standalone and print the VIN directly.

In [4]:
# print(f"[CAN Test] VIN: {get_vin_from_can()}")